# Lemmatisierung mit regulären Ausdrücken ... zumindest fast
In dieser Übung geht es darum, die Flexionsaffixe von Verben zu entfernen. Dies ist ein erster Schritt zur Lemmatisierung, also dazu, konkrete Wortformen auf ihre Zitierformen im Lexikon zurückzuführen. Wir werden dabei sehen, dass es mit diesem Schritt oft nicht getan ist, dass ordentliche Lemmatisierung also noch ein gutes Stück schwieriger ist.

## Nötige Pakete importieren
Um eine TSV-Datei zu öffnen, verwenden wir hier *pandas*, weil wir damit leichter auf einzelne Tabellenspalten zuzugreifen. Achtung, Pandas ist nicht Teil der Standardbibliothek und muss separat installiert werden. Das sollte in den meisten Python3-Umgebungen über den Python-eigenen Paketmanager `pip3` möglich sein. Um Pandas zu installieren, muss nur der Befehl `pip3 install --user pandas` in einem Terminal ausgeführt werden. (Wer eine Entwicklungsumgebung wie PyCharm nutzt, kann das ggf. auch über ein Menü dieser Umgebung erledigen.)

Sollte die Installation von Pandas nicht möglich sein, kann die TSV-Datei auch über das `csv`-Modul der Standardbibliothek eingelesen werden. Eine Anleitung, wie das möglich ist, gibt es z. B. [hier](https://medium.com/@adds68/parsing-tsv-file-with-csv-in-python-662d6347b0cd).

Außerdem brauchen wir natürlich reguläre Ausdrücke.

In [1]:
import pandas as pd
import re

## Tabelle einlesen
Die Datei "uebung_verben.tsv" enthält 500 Treffer aus einer Suche nach Vollverben im DWDS-Kernkorpus. (Die Abfrage sieht so aus: <https://www.dwds.de/r?q=%24p%3DVV*+%23sep&corpus=kern&date-start=1900&date-end=1999&genre=Belletristik&genre=Wissenschaft&genre=Gebrauchsliteratur&genre=Zeitung&format=full&sort=random&limit=10>)

Neben den konkreten Wortformen, die von der Abfrage gefunden wurden, enthält die Datei auch für jede Form den kompletten Satz, in dem sie vorkommt, sowie Informationen zur Textquelle. Jede Information befindet sich in einer eigenen Spalte der Tabelle.

Da Spalten in dieser Tabelle mit Tabstops (`\t`) getrennt sind, bietet sich `read_table()` an.

In [2]:
path = "data/uebung_verben.tsv"
table = pd.read_table(path)
table

,match,sentence,collection,author,title,textClass,bibl,date
0,entsteht,"Es bleibt wohl dem , der die Frage zu stellen ...",kern,"Sloterdijk, Peter",Kritik der zynischen Vernunft,Wissenschaft,"Sloterdijk, Peter: Kritik der zynischen Vernun...",1983-12-31
1,verloren,"Diese Sumpfreminiszenzen , die mich nicht herg...",kern,"May, Karl",Mein Leben und Streben,Gebrauchsliteratur::Autobiographien,"May, Karl: Mein Leben und Streben, Selbstbiogr...",1910-12-31
2,abschaffen,Hitler ist schon seit langen Monaten deutscher...,kern,Kurt Tucholsky:Kaspar Hauser,Hitler und Goethe,Zeitung,"Tucholsky, Kurt: Hitler und Goethe. In: Kurt T...",1932-05-17
3,hielt,"Warum tat er es nicht , sondern hielt an sich ...",kern,"Huch, Ricarda",Der große Krieg in Deutschland,Belletristik::Gebrauchsliteratur,"Huch, Ricarda: Der Dreißigjährige Krieg, Wiesb...",1914-12-31
4,bewahren,"Er besitzt eigentlich alles , was eine Frau vo...",kern,"Strauß, Botho",Der junge Mann,Belletristik::Roman,"Strauß, Botho: Der junge Mann, München: Hanser...",1984-12-31
...,...,...,...,...,...,...,...,...
495,fälle,Und der Satz » ich fälle einen Baum « ist eben...,kern,"Mauthner, Fritz",Wörterbuch der Philosophie,Wissenschaft::Phiosophie,"Mauthner, Fritz: Wörterbuch der Philosophie. I...",1910-12-31
496,lebte,Für mich war der Kreis bedeutsam durch eine st...,kern,"Steiner, Rudolf",Mein Lebensgang,Gebrauchsliteratur::Autobiographien,"Steiner, Rudolf: Mein Lebensgang, Eine nicht v...",1925-04-05
497,wagte,"Bald wagte ein Wärter , sich ihm und der Kiste...",kern,"Morgner, Irmtraud",Leben und Abenteuer der Trobadora Beatriz nach...,Belletristik,"Morgner, Irmtraud: Leben und Abenteuer der Tro...",1974-12-31
498,abbauen,Und wenn die Ausgaben hierfür und für die Wart...,kern,"Grimm, Hans",Volk ohne Raum,Belletristik::Roman,"Grimm, Hans: Volk ohne Raum, München: Langen 1...",1926-12-31


## Verbformen

Wir brauchen hier eigentlich nur die erste Spalte ...

In [3]:
verben = table["match"].sort_values()
verben

185    Ausgeschlossen
422              Geht
343              Weiß
369         Wiederhol
280            Ziehen
            ...      
89         überreicht
438       überschätzt
442         übersehen
345          übertraf
357         überzeugt
Name: match, Length: 500, dtype: object

Das ist nun strenggenommen ein spezieller Objekttyp aus *pandas* (nämlich *Series*), aber wir können ganz normal damit arbeiten.

Die Aufgabe besteht nun darin, den untenstehenden Code so anzupassen, dass bei jeder Verbform Flexionsaffixe entfernt werden.  Eine kurze Übersicht über Verbkonjugation im Deutschen gibt es z.B. bei Wikipedia: <https://de.wikipedia.org/wiki/Konjugation_(Grammatik)>

Finde also einen passenden regulären Ausdruck!

Welche Probleme treten dabei auf? Welche weiteren Schritte müssten für eine vollständige Lemmatisierung noch umgesetzt werden?

**Größte verbleibende Probleme: Umlaute und unregelmäßige Formen! Sind nicht vorhersehbar und müssen in einem Lexikon nachgeschlagen werden.**

In [4]:
infinitives = []
flektionsmuster = [
    r'(\w+)end', # partizip 1
    r'(\w+)ge(\w+)(?:t|en)', # perfekt partikelverb
    r'ge(\w+)(?:t|en)', # perfekt
    r'(\w+[^t])t(?:en|[se]t|[et])', # präteritum
    r'(\w+)(?:en|[se]t|[etd])', # präsens
    r'(\w+[rl])n', # infinitiv
    r'(\w+)' # catch-all
]

# nicht gefordert, aber auch interessant. Stichwort 2-Ebenen-Morphologie.
tilgungen = [
    (r"((?<!i)e[rl])'e", r"\1"),
    (r"e'e", "e"),
    (r"([aeiouäöü])'e", r"\1te"),
    (r"'", "")
]

def lemma(verb):
    for muster in flektionsmuster:
        match = re.fullmatch(muster, verb)

        if match is not None:
            return ''.join(match.groups())

def tilgung(lemma):
    for muster, repl in tilgungen:
        lemma = re.sub(muster, repl, lemma)
    return lemma

for verb in verben:
    zwischen = lemma(verb) + "'en"
    infinitiv = tilgung(zwischen)

    print(f"{verb} -> {zwischen} -> {infinitiv}")
    
    infinitives.append(infinitiv)

Ausgeschlossen -> Ausschloss'en -> Ausschlossen
Geht -> Geh'en -> Gehen
Weiß -> Weiß'en -> Weißen
Wiederhol -> Wiederhol'en -> Wiederholen
Ziehen -> Zieh'en -> Ziehen
abbauen -> abbau'en -> abbauten
abgeben -> abb'en -> abben
abgegeben -> abgeb'en -> abgeben
abgelehnt -> ablehn'en -> ablehnen
abgeschafft -> abschaff'en -> abschaffen
abgeändert -> abänder'en -> abändern
ablegte -> ableg'en -> ablegen
abschaffen -> abschaff'en -> abschaffen
abschnitten -> abschnitt'en -> abschnitten
abspielen -> abspiel'en -> abspielen
abstammen -> abstamm'en -> abstammen
abwenden -> abwend'en -> abwenden
anerkannt -> anerkann'en -> anerkannen
angedeihen -> andeih'en -> andeihen
angekettet -> ankette'en -> anketten
angeredet -> anrede'en -> anreden
angerichtet -> anrichte'en -> anrichten
angesehen -> anseh'en -> ansehen
angewachsen -> anwachs'en -> anwachsen
ankommt -> ankomm'en -> ankommen
anließen -> anließ'en -> anließen
anpaßte -> anpaß'en -> anpaßen
ansammeln -> ansammel'en -> ansammeln
anschauen ->

### Evaluation
Die folgende Methode überprüft, wie hoch der Anteil der korrekt lemmatisierten Worte ist. Evaluiere gerne einmal deine Lösung und versuche ein von Dir gestecktes Ziel, wie etwa 65% korrekter Lemmata, zu erreichen.

In [5]:
def evaluate(infinitives, method='accuracy'):
    """Evaluate the accuracy of a list of infinitives against their lemma.

    Parameters
    ----------
    infinitives : iterable of str
        The infinitives to be evaluated.
    method : {'accuracy', 'positives', 'negatives'}
        Wether to return only the accuracy of lemmatisation or 
        additionally print either all correct or incorrect infinitives to 
        the console.
        
    Returns
    -------
    float
        The percentage of correctly lemmatised infinitives.

    """
    def compare_pair(word1, word2, method):
        if word1 == word2:
            if method == 'positives':
                print(word1)
            return 1
        
        elif method == 'negatives':
            print(word1, word2)

        return 0
        
    correct = 0
    total = 0
    
    with open('data/uebung_verben_lemmata.txt', encoding='utf-8') as lemmata:
        for result, lemma in zip(infinitives, lemmata):
            total += 1
            correct += compare_pair(result, lemma.rstrip(), method)
        
    return correct / total

In [6]:
evaluate(infinitives)

0.674